#Подготовка

In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/1050931/train_dataset_train.zip
#!cp /content/drive/MyDrive/Factoryvision/train_dataset_train.zip .
!unzip -q ./train_dataset_train.zip
!mkdir data
!mkdir ./data/train/
!mkdir ./data/valid/
!mkdir data/train/images
!mkdir data/valid/images
!mkdir data/train/labels
!mkdir data/valid/labels
!mv cameras data
!mv danger_zones data
!touch pothole_v8.yaml
!echo "path: './data/'" >> pothole_v8.yaml
!echo "train: 'train/images'" >> pothole_v8.yaml
!echo "val: 'valid/images'" >> pothole_v8.yaml
!echo "names:" >>  pothole_v8.yaml
!echo "  0: 'person'" >> pothole_v8.yaml
!pip install --upgrade ultralytics==8.0.186 wandb shapely dill

In [48]:
import pandas as pd
import json
import os
import re
import numpy as np
#import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch
import random
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from copy import deepcopy
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR
#from google.colab.patches import cv2_imshow
from shapely.geometry import Polygon, box
import wandb
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import cv2


In [29]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed()

Random seed set as 42


#Данные

In [30]:
data = []
for i in os.listdir("./data/cameras"):
  for j in  os.listdir(f"./data/cameras/{i}"):
    if j.split('.')[-1] == "jpg":
      data.append(f"./data/cameras/{i}/{j}")
    elif j.split('.')[-1] == "Png":
      data.append(f"./data/cameras/{i}/{j}")

In [31]:
train, test = train_test_split(data, test_size=0.1, random_state=int(os.environ["PYTHONHASHSEED"]))

In [32]:
for i in train:
  rash = i[-3:]
  a = i[:-4]
  try:
   if rash == "jpg":
       os.rename(f"{a}.jpg", f"./data/train/images/{a.split('/')[-2]}&{a.split('/')[-1]}.jpg")
   else:
       os.rename(f"{a}.Png", f"./data/train/images/{a.split('/')[-2]}&{a.split('/')[-1]}.Png")
   os.rename(f"{a}.txt", f"./data/train/labels/{a.split('/')[-2]}&{a.split('/')[-1]}.txt")
  except Exception as e:
    print(e, i)

In [33]:
for i in test:
  rash = i[-3:]
  a = i[:-4]
  try:
   if rash == "jpg":
       os.rename(f"{a}.jpg", f"./data/valid/images/{a.split('/')[-2]}&{a.split('/')[-1]}.jpg")
   else:
       os.rename(f"{a}.Png", f"./data/valid/images/{a.split('/')[-2]}&{a.split('/')[-1]}.Png")
   os.rename(f"{a}.txt", f"./data/valid/labels/{a.split('/')[-2]}&{a.split('/')[-1]}.txt")
  except Exception as e:
    print(e, i)

#Обучение

In [ ]:
EPOCHS = 300
BATCH = 16
IMGSZ = 1280
NAME = "yolov8n"
MODEL = 'yolov8n.pt' # n s m l - пробуем все

In [ ]:
wandb.login(key="2dffb26d8578c9b3cd196ec554c5f856d28c1dfa")
wandb.init(
    project="yolov8fin")

In [ ]:
model = YOLO(MODEL)
add_wandb_callback(model, enable_model_checkpointing=True)
results = model.train(
   data='pothole_v8.yaml',
   imgsz=IMGSZ,
   epochs=EPOCHS,
   batch=BATCH,
   name=NAME,
   project = 'yolov8fin',
   )
model.val()
wandb.finish()

#Вспомогательные функции

In [58]:
def intersectionArea(poly_coords, square_coords):
    poly = Polygon(poly_coords)
    square = box(*square_coords)
    intersection = poly.intersection(square)
    if intersection.area == 0:
      return (False, 0)
    p =  intersection.area / square.area
    return (p >= 0.15, round(p, 2))

In [11]:
def giveDictPolygon():
  d = {}
  for i in os.listdir("./data/danger_zones"):
    if i.split('.')[-1] == "txt":
      with open(f"./data/danger_zones/{i}") as r:
        data = r.read()
        name = i.split("_")[1].split('.')[0]
        if name not in d:
          d[name] = []
        d[name].append([])
        for j in data.split(",\n"):
          d[name][-1].append(list(map(int, j.strip()[1:-1].split(", "))))
  return d

#ИНФЕРЕНС

In [34]:
CONF_YOLO = 0.45
MODEL_NAME = "yolov8l.pt" # models/yolov8s.pt или models/yolov8l.pt Пропишите путь до модели, они находятся в папку models в репозитории
model = YOLO(MODEL_NAME)

In [65]:
def imageanswer(image, cam_name, model, conf_yolo):

    intersectionList = []

    predict_yolo = model.predict(
        source=image, conf=conf_yolo, verbose=False
    )
    image = cv2.imread(image)
    for i in giveDictPolygon()[cam_name]:
        for j in range(len(i) - 1):
            cv2.line(image, i[j], i[j + 1], (0, 255, 0), 3)
        cv2.line(image, i[-1], i[0], (0, 255, 0), 3)

    coors = predict_yolo[0].boxes.data

    for i in coors:
        coor = list(map(int, list(i)[:-2]))
        cv2.rectangle(image, (coor[0], coor[1]), (coor[2], coor[3]), (0, 255, 255), 2)
        intersectionListTemp = []
        for j in giveDictPolygon()[cam_name]:
            intersectionListTemp.append(intersectionArea(j, coor))
        intersectionList.append(max(intersectionListTemp, key=lambda x: x[1]))
        if len(intersectionList) == 0:
          return [[[False, 0]], image]
    return intersectionList, image

In [7]:
def makeCSV(path):
  df = pd.read_csv(path)
  df.to_csv("submit.csv", index=False)

In [ ]:
image = "example/DpR-Csp-uipv-ShV-V15d56b754-3f8a-4665-8b2e-1d0b340be4bb.jpg"
cam_name = "DpR-Csp-uipv-ShV-V1"
#image =  "example/Pgp-lpc2-K-0-1-38&Pgp-lpc2-K-0-1-38_Nepreryvnaya_podgruppa_(kleti_3-5)_2021-11-17_15_40_09.481.Png"
#cam_name = "Pgp-lpc2-K-0-1-38"
#image = "example/Php-Angc-K3-8&Php-Angc-K3-8_angc4test1144.jpg"
#cam_name = "Php-Angc-K3-8"


In [ ]:
data = MLflex.imageanswer(image, cam_name, model, conf_yolo)
print(data[0])
cv2.imwrite('example.jpg', data[1])